In [1]:
import module.TorchDataset as td
import platform
import importlib

importlib.reload(td) # 클래스 수정 시 import 리로드

# os에 따라 다르게 적용 (코랩은 리눅스)
current_os = platform.system()
sep = "\\" if current_os == "Windows" else "/" 

img_size = 224

# TestSet
root_test = f"..{sep}01.Data{sep}fire{sep}test{sep}" # 루트폴더
extension = "jpg" # 찾을 확장자

test_dataset = td.TorchDataset(root_test, sep, extension, img_size, "test")

print(len(test_dataset))
print(test_dataset.get_label_dict())

6299
{'nowildfire': 0, 'wildfire': 1}


In [2]:
import module.TorchDatasetCNN as tdCNN
importlib.reload(tdCNN)

cnn_runner = tdCNN.TorchDatasetCNN()
batch_size = 20

test_dataloader = cnn_runner.get_dataloader(test_dataset, batch_size, True, False)
len(test_dataloader)

315

In [3]:
######################## Ensenble CNN #####################
###########################################################
###########################################################
###########################################################

In [8]:
import torch.nn as nn
from torchvision import models

device = cnn_runner.get_device()

custom_save_path = ""
resnet50_layer4_save_path = ""

if current_os == "Windows": # 로컬 저장용
    custom_save_path = f"save_model{sep}03.Torch_CNN{sep}02.9_layer_Custom_CNN{sep}fire_custom_9_CNN.pth"
    resnet50_layer4_save_path = f"save_model{sep}03.Torch_CNN{sep}05.resnet50_Layer4_Finetune{sep}resnet50_Layer4_Finetune.pth"
    resnet50_full_save_path = f"save_model{sep}03.Torch_CNN{sep}04.resnet50_Full_Finetune{sep}resnet50_Full_Finetune.pth"
else:
    custom_save_path = f"..save_model{sep}fire_custom_9_CNN.pth"
    resnet50_layer4_save_path = f"..save_model{sep}resnet50_Layer4_Finetune.pth" # 코랩 저장용
    resnet50_full_save_path = f"..save_model{sep}resnet50_Full_Finetune.pth.pth"

In [9]:
import module.CNN as CNN
importlib.reload(CNN)

# Custom CNN 로드
custom_model = CNN.CNN(img_size).to(device)
custom_model = cnn_runner.load_model(custom_model, custom_save_path, device)
custom_model.to(device)

# ResNet layer4 Finetune CNN 로드
import torch.nn as nn
from torchvision import models
resnet_model = models.resnet50().to(device)

# fc layer 수정
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 2)
resnet_model = cnn_runner.load_model(resnet_model, resnet50_layer4_save_path, device)
resnet_model = resnet_model.to(device)

# ResNet full Finetune CNN 로드
import torch.nn as nn
from torchvision import models
resnet_full_model = models.resnet50().to(device)

# fc layer 수정
resnet_full_model.fc = nn.Linear(resnet_full_model.fc.in_features, 2)
resnet_full_model = cnn_runner.load_model(resnet_full_model, resnet50_full_save_path, device)
resnet_full_model = resnet_full_model.to(device)


In [6]:
# 앙상블 모델 테스트
kind = "custom_9, resnet50_4_F"
cnn_runner.test_ensenble_model([custom_model, resnet_model], test_dataloader, device, kind)

100%|██████████| 315/315 [15:47<00:00,  3.01s/it]

custom_9, resnet50_4_F
일치 : 6257 / 6299, 정확도 : 99.33
리콜 = tp : 3463, fn : 16 / racall : 99.54


In [10]:
# 앙상블 모델 테스트
kind = "resnet50_4_F, resnet50_Full"
cnn_runner.test_ensenble_model([custom_model, resnet_model, resnet_full_model], test_dataloader, device, kind)

100%|██████████| 315/315 [24:49<00:00,  4.73s/it]

resnet50_4_F, resnet50_Full
일치 : 6254 / 6299, 정확도 : 99.29
리콜 = tp : 3466, fn : 13 / racall : 99.63
